## CA 2 - Big Data & Advanced Analytics

### Using Pyspark for data exploration

In [1]:
# clear the cache on the spark session
spark.catalog.clearCache()

In [2]:
# what version of pyspark is running on the computer using SparkContext
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
# sc master - running locally
sc.master

'local[*]'

In [4]:
# Import regex module
import re
from operator import add

# Import Pyspark
import pyspark
from pyspark.sql import SparkSession

# Other Libraries 
import findspark
findspark.init()

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

#### Import File from the Hadoop Directory

<b>References</b>
1. https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/
2. https://medium.com/@ashutoshkumar2048/spark-connect-apache-spark-3-4-9846c40484d0
3. 

In [5]:
# start the spark session for CA2

spark = SparkSession.builder.appName("ca2").getOrCreate()

In [6]:
# Create a structure to hold the data, name and define data types
schema = StructType([
    StructField("target", IntegerType(), True),
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("query", StringType(), True),
    StructField("author", StringType(), True),
    StructField("tweet", StringType(), True)])


In [7]:
# Read input file from hadoop directory on the local drive
# import the csv file from hadoop
path = "/user1/twitter_DS_1yr.csv"

df = spark.read.csv(path, header=False, inferSchema=True, schema=schema)


In [8]:
# Import check - show the first 10 rows of the imported table

#df.show(10)

#### Exploring the dataframe

In [9]:
# Get the number of rows
num_rows = df.count()

# Get the number of columns
num_columns = len(df.columns)

# Print the shape
print("Number of rows: ", num_rows)
print("Number of columns: ", num_columns)

Number of rows:  1600000
Number of columns:  6


In [10]:
# look at types of values in the polarity

print(f"There is {df[df['target']==4].count()} positive values in the dataframe.")

print(f"There is {df[df['target']==0].count()} negative values in the dataframe.")

There is 800000 positive values in the dataframe.


There is 800000 negative values in the dataframe.


In [11]:
# drop duplicate entries

df = df.dropDuplicates()

#### Feature Engineering

In [12]:
# import relevant libaries

from pyspark.sql.functions import udf, regexp_replace, lower
from pyspark.sql.types import StringType


In [13]:
# drop any n/a rows

df.dropna()

DataFrame[target: int, id: string, date: string, query: string, author: string, tweet: string]

In [14]:
# remove noise such as html links, stop words / punctuation / #hashtags etc
# REFERENCE: https://medium.com/towards-artificial-intelligence/large-scale-sentiment-analysis-with-pyspark-bdccf9256e35

def pre_process(text):
    # Remove links
    #text = re.sub('http://\S+|https://\S+', '', text)
    #text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r'http\S+', '', text)

    # Convert HTML references
    text = re.sub(r'&amp', 'and', text)
    text = re.sub(r'&lt', '<', text)
    text = re.sub(r'&gt', '>', text)
    #text = re.sub(' ', text)

    # Remove new line characters
    text = re.sub(r'[\r\n]+', ' ', text)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove multiple space characters
    text = re.sub(r'\s+',' ', text)
    
    # Convert all text to lowercase
    text = text.lower()
    
    return text

In [15]:
# Register the pre_process function as a UDF (User-Defined Function)

pre_process_udf = udf(pre_process, StringType())

In [16]:
# Apply the UDF to the 'Tweet_details' column and create a new column 'Processed_tweet_details'

df = df.withColumn('processed_tweet', pre_process_udf('tweet'))

In [17]:
# drop duplicate entries

df = df.dropDuplicates()

In [18]:
# Show the DataFrame with the new column

df.show(10)

+------+----------+--------------------+--------+---------------+--------------------+--------------------+
|target|        id|                date|   query|         author|               tweet|     processed_tweet|
+------+----------+--------------------+--------+---------------+--------------------+--------------------+
|     0|1468106999|Mon Apr 06 23:45:...|NO_QUERY|    stuartheron|@watko Shockingly...|    shockingly not! |
|     0|1686953810|Sun May 03 07:19:...|NO_QUERY|forevaguitargrl|up ad ready for c...|up ad ready for c...|
|     0|1956161267|Thu May 28 21:18:...|NO_QUERY|      SupaSista|Someone just sent...|someone just sent...|
|     0|1956910895|Thu May 28 23:00:...|NO_QUERY| jeanettiewuvsu|Wtf is wrong with...|wtf is wrong with...|
|     0|1963373312|Fri May 29 12:37:...|NO_QUERY|        XkyRauh|Only two weeks le...|only two weeks le...|
|     0|1972460907|Sat May 30 09:16:...|NO_QUERY|         jiaaaa|You gave me false...|you gave me false...|
|     0|1973029187|Sat May 3

#### Feature Extraction

<b>References</b>
1. https://medium.com/towards-artificial-intelligence/large-scale-sentiment-analysis-with-pyspark-bdccf9256e35

2. https://medium.com/@chris_42047/an-easy-guide-to-basic-twitter-sentiment-analysis-python-tutorial-1630d5213ff6

3. https://www.kaggle.com/code/muhammetzahitaydn/pyspark-sentiment-analysis-with-word2vec-embedding


In [19]:
# Import the relevant libraries tocreate a piplelin

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StringIndexer, CountVectorizer, NGram, VectorAssembler, ChiSqSelector



In [20]:
# create a tokenizer 

tokenizer = Tokenizer(inputCol="processed_tweet", outputCol="words")

In [21]:
# HashingTF: Hashing Term Frequency
# REFERENCE - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.HashingTF.html#pyspark.ml.feature.HashingTF
# Maps a sequence of terms to their term frequencies using the hashing

hashtf = HashingTF(inputCol="words", outputCol='tf')

In [22]:
# pass the hashtf function to the IDF function
# REFERENCE - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.IDF.html#pyspark.ml.feature.IDF
# Compute the Inverse Document Frequency (IDF) given a collection of documents ie the tweets df

idf = IDF(inputCol='tf', outputCol="features")

In [23]:
# Index labels

label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")

In [24]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx, lr])

#### Filter data based on key word - 'weather'

##### REFERENCES 
1. https://towardsdatascience.com/sentiment-analysis-with-pyspark-bc8e83f80c35
2. https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [25]:
# create a variable to search within the dataframe on

search_for = "weather"


In [26]:
df2 = df.filter(df["processed_tweet"].contains(search_for))

In [27]:
df2.show(10)

+------+----------+--------------------+--------+---------------+--------------------+--------------------+
|target|        id|                date|   query|         author|               tweet|     processed_tweet|
+------+----------+--------------------+--------+---------------+--------------------+--------------------+
|     4|1964587188|Fri May 29 14:28:...|NO_QUERY|       SuperRob|@sparktography We...| weather.com says...|
|     0|2264778973|Sun Jun 21 04:55:...|NO_QUERY|    deirdre7194|@Es94 yay!! i did...| yay!! i didnt no...|
|     4|2060752623|Sat Jun 06 19:30:...|NO_QUERY|     HailHorror|Finishing my pack...|finishing my pack...|
|     0|2229093418|Thu Jun 18 15:16:...|NO_QUERY| nocheapthrillz|@afreshmusic hey ...| hey d! my bad, i...|
|     0|2191448262|Tue Jun 16 05:17:...|NO_QUERY|     tippielove|PS for those intr...|ps for those intr...|
|     0|2222121192|Thu Jun 18 06:31:...|NO_QUERY|  abitheamazing|so its nice weath...|so its nice weath...|
|     0|1992186304|Mon Jun 0

In [28]:
# Get the number of rows in the new df
num_rows = df2.count()

# Get the number of columns
num_columns = len(df2.columns)



In [29]:
# Print the shape
print("Number of rows: ", num_rows)
print("Number of columns: ", num_columns)

Number of rows:  10342
Number of columns:  7


### **************************************************************



#### Sentiment extraction using TextBlob

##### REFERENCES

1. https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524
2. https://towardsdatascience.com/sentiment-analysis-with-pyspark-bc8e83f80c35
3. https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [30]:
from pyspark.sql import SparkSession
from textblob import TextBlob


In [31]:
# Define a UDF to apply sentiment analysis using TextBlob

def get_sentiment(processed_tweet):
    analysis = TextBlob(processed_tweet)
    
    sentiment = analysis.sentiment.polarity

    return sentiment



In [32]:

# Register the UDF
get_sentiment_udf = spark.udf.register("get_sentiment", get_sentiment)

In [33]:
# Apply the UDF to the 'tweet' column and create a new column 'sentiment'
df = df.withColumn('sentiment', get_sentiment_udf('processed_tweet'))


In [34]:

# Show the DataFrame with the 'tweet' and 'sentiment' columns
df.select('processed_tweet', 'sentiment').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|processed_tweet                                                                                                                                    |sentiment          |
+---------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
| shockingly not!                                                                                                                                   |0.5                |
|up ad ready for church, gotta start packing today... really starting to think about how much i'm gonna miss ethan this summer.                     |0.13333333333333333|
|someone just sent a text message, but i can't see it.                                                                                              |0

#### Save Weather Data to a new Dataframe

In [38]:
# REFERENCE - https://sparkbyexamples.com/pyspark/pyspark-write-dataframe-to-csv-file/

#rename the dataframe

weather_tweets2 = df2

In [40]:
# save the file to a csv file
weather_tweets2.write.csv("hdfs://localhost:9000/user1/weather_tweets2")

2023-05-21 17:54:52,866 WARN hdfs.DataStreamer: Caught exception  (2 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.closeInternal(DataStreamer.java:847)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:843)
2023-05-21 17:54:53,135 WARN hdfs.DataStreamer: Caught exception  (3 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2023-05-21 17:54:53

2023-05-21 17:55:02,156 WARN hdfs.DataStreamer: Caught exception (49 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2023-05-21 17:55:02,309 WARN hdfs.DataStreamer: Caught exception (51 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.closeInternal(DataStreamer.java:847)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:843)
2023-05-21 17:55:02

2023-05-21 17:55:12,436 WARN hdfs.DataStreamer: Caught exception(115 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.closeInternal(DataStreamer.java:847)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:843)
2023-05-21 17:55:12,544 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2023-05-21 17:55:12,797 WARN hdfs.D

2023-05-21 17:55:20,674 WARN hdfs.DataStreamer: Caught exception(176 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2023-05-21 17:55:20,821 WARN hdfs.DataStreamer: Caught exception(177 + 1) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2023-05-21 17:55:21,109 